In [7]:
# --------- import required functions

from tensorflow.keras.layers import *
import pandas as pd



from tensorflow.keras.layers import *
from glob import glob
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np
from tensorflow.keras.utils import CustomObjectScope



In [8]:
# -------------dice and dice_loss 
def dice_coef(y_true, y_pred, smooth=1):
    y_true = K.cast(y_true, 'float64')
    y_pred = K.cast(y_pred, 'float64')
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union        = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice         = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

def dice_loss(y_true, y_pred):
  return 1.-dice_coef(y_true, y_pred)


# ----------build_unet  
class CONV:
    def conv_block(data, base, batchnorm=False):
        # --------------- 1st block
        x = Conv2D(filters=base, kernel_size=(3, 3), padding='same')(data)
        if batchnorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)
        # --------------- 2nd block
        x = Conv2D(filters=base, kernel_size=(3, 3), padding='same')(x)
        if batchnorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x

def build_unet(h, w, c, base, classes, dropout=True, batchnorm=True):
    num_filters = [base, base * 2, base * 4, base * 8]
    inputs = Input((h, w, c))

    skip_x = []
    x = inputs

    # ----------------- Encoder
    for f in num_filters:
        x = CONV.conv_block(x, f, batchnorm=True)
        skip_x.append(x)
        x = MaxPooling2D(pool_size=(2, 2), padding="same")(x)
        if dropout:
            x = Dropout(0.2)(x)

            # ----------------- Bridge
    x = CONV.conv_block(x, num_filters[-1], batchnorm=True)
    # x = MaxPooling2D(pool_size=(2, 2), padding="same") (x)  # no need to this layer
    if dropout:
        x = Dropout(0.2)(x)

    num_filters.reverse()
    skip_x.reverse()

    # ----------------- Decoder
    for i, f in enumerate(num_filters):
        #x = UpSampling2D((2, 2))(x)  # Conv2DTranspose(f, (2,2)) (x)
        x = Conv2DTranspose(filters=f, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
        xs = skip_x[i]
        #height, width = determine_crop(xs, x)
        #crop_xs = Cropping2D(cropping=(height, width))(xs)
        x = Concatenate()([x, xs])
        if dropout:
            x = Dropout(0.2)(x)
        x = CONV.conv_block(x, f, batchnorm=True)

    # ----------------- Output
    out = Conv2D(classes, (1, 1), padding="same", activation='sigmoid')(x)

    # ----------------- input and output
    model = Model(inputs=inputs, outputs=out)
    model.summary()
    return model




In [33]:
modelFilePath = "./"
mainPath="I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset"
workDir = "images"
outDir = "cutOffByMaskOutput"
workPath = os.path.join(mainPath,workDir)
outPath = os.path.join(mainPath,outDir)

In [90]:
model = tf.keras.models.load_model(os.path.join(modelFilePath, 'model.h5'), compile=False)
model.compile() 

In [71]:
def load_data(ipath):
    pattern_ipath = ipath + "/*" 

    images = sorted(glob(os.path.join(pattern_ipath)))

    images = np.array(images)

    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)

    images = images[indices]

    return images.tolist()

def read_image(path, H=912, W=912):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W,H))
    x = x/255.0
    x = x.astype(np.float32)
    return x



def preprocess(x,H=912, W=912):

    def f(x):
        print(x)
        x = x.decode()

        x = read_image(x)

        return x

    images = tf.numpy_function(f, [x], [tf.float32])
    #print(images)
       #images.set_shape([H, W, 1])

    return images

AUTOTUNE = tf.data.AUTOTUNE

def tf_data(x):
    data = tf.data.Dataset.from_tensor_slices((x))
    data = data.cache()
    #data = data.shuffle(buffer_size=1000)
    data = data.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data = data.batch(1)
    data = data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return data


In [72]:
fileList = load_data(workPath)
dataset_ = tf_data(fileList)
for z in dataset_:
    print(z)

b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SS6114_batch2_white_ok_1.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN6937_batch2_white_ok_0.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SS5906_batch2_white_ok_2.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SS6272_batch2_white_ok_1.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN4778_batch2_white_ok_0.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SS6503_batch2_white_ok_0.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG975_batch2_brown_ok_0.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG1262_batch2_brown_ok_0.jpg'
b'I:\\download\\deepLearn

(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.2627451 , 0.24705882, 0.08627451, ..., 0.11372549,
         0.13725491, 0.10980392],
        [0.25882354, 0.2509804 , 0.1254902 , ..., 0.10980392,
         0.14901961, 0.13333334],
        [0.27058825, 0.24705882, 0.09411765, ..., 0.05882353,
         0.10196079, 0.10588235],
        ...,
        [0.2627451 , 0.23921569, 0.09411765, ..., 0.99607843,
         1.        , 1.        ],
        [0.24705882, 0.23137255, 0.07058824, ..., 1.        ,
         1.        , 1.        ],
        [0.24705882, 0.24313726, 0.06666667, ..., 0.8392157 ,
         0.8156863 , 0.77254903]]], dtype=float32)>,)
(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.29411766, 0.2784314 , 0.17254902, ..., 0.10588235,
         0.04313726, 0.        ],
        [0.25490198, 0.23137255, 0.1254902 , ..., 0.07843138,
         0.04313726, 0.        ],
        [0.25490198, 0.23137255, 0.11764706, ..., 0.07058824,
         0.08235294, 0.02

(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.25490198, 0.23529412, 0.08235294, ..., 0.09019608,
         0.14117648, 0.13333334],
        [0.2901961 , 0.27058825, 0.14117648, ..., 0.08627451,
         0.14117648, 0.09411765],
        [0.2784314 , 0.23529412, 0.08627451, ..., 0.07058824,
         0.10980392, 0.09803922],
        ...,
        [0.4509804 , 0.36078432, 0.14117648, ..., 0.23921569,
         0.23921569, 0.14117648],
        [0.43137255, 0.33333334, 0.15294118, ..., 0.15294118,
         0.15686275, 0.11372549],
        [0.4117647 , 0.31764707, 0.17254902, ..., 0.14901961,
         0.14509805, 0.10980392]]], dtype=float32)>,)b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN4926_batch2_white_ok_0.jpg'

(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.42352942, 0.38039216, 0.24705882, ..., 0.16862746,
         0.16862746, 0.17254902],
        [0.5254902 , 0.47058824, 0.3529412 , ..., 0.2627451 ,

b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN4582_batch2_white_ok_0.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG409_batch2_brown_ok_1.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN4843_batch2_white_ok_0.jpg'
(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.25882354, 0.21568628, 0.10196079, ..., 0.09803922,
         0.09803922, 0.01960784],
        [0.23921569, 0.19215687, 0.08235294, ..., 0.07843138,
         0.07843138, 0.02352941],
        [0.23529412, 0.19215687, 0.07058824, ..., 0.10980392,
         0.10980392, 0.07058824],
        ...,
        [0.07843138, 0.07450981, 0.03137255, ..., 0.03529412,
         0.05882353, 0.0627451 ],
        [0.09803922, 0.09019608, 0.04705882, ..., 0.10588235,
         0.14117648, 0.13333334],
        [0.09803922, 0.08235294, 0.02745098, ..., 0.05882353,
         0.11372549, 0.0

(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.17254902, 0.14509805, 0.05882353, ..., 0.05098039,
         0.06666667, 0.04313726],
        [0.17254902, 0.14509805, 0.05490196, ..., 0.03921569,
         0.05882353, 0.04705882],
        [0.1882353 , 0.15686275, 0.0627451 , ..., 0.05882353,
         0.07058824, 0.04705882],
        ...,
        [0.06666667, 0.07450981, 0.03529412, ..., 0.05490196,
         0.08235294, 0.05882353],
        [0.11372549, 0.12156863, 0.07843138, ..., 0.09803922,
         0.10980392, 0.09019608],
        [0.11372549, 0.11764706, 0.05882353, ..., 0.05490196,
         0.06666667, 0.02352941]]], dtype=float32)>,)
(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.29803923, 0.3254902 , 0.17254902, ..., 0.08627451,
         0.16078432, 0.13333334],
        [0.3647059 , 0.3882353 , 0.27450982, ..., 0.18431373,
         0.23921569, 0.21176471],
        [0.3372549 , 0.3137255 , 0.1764706 , ..., 0.15686275,
         0.19215687, 0.19

...

(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.24705882, 0.20392157, 0.06666667, ..., 0.01960784,
         0.01960784, 0.01960784],
        [0.25490198, 0.20392157, 0.07058824, ..., 0.03921569,
         0.04313726, 0.03921569],
        [0.25490198, 0.21176471, 0.07058824, ..., 0.04705882,
         0.0627451 , 0.03529412],
        ...,
        [0.38431373, 0.3254902 , 0.2       , ..., 0.28235295,
         0.25490198, 0.14901961],
        [0.44705883, 0.39215687, 0.2627451 , ..., 0.36862746,
         0.34509805, 0.32156864],
        [0.45490196, 0.39607844, 0.23137255, ..., 0.29411766,
         0.28235295, 0.28235295]]], dtype=float32)>,)b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SS6310_batch2_white_ok_1.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG1529_batch2_brown_ok_3.jpg'

(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.3254902 , 0.30980393, 0.

b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG1547_batch2_brown_ok_1.jpg'b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG888_batch2_brown_ok_4.jpg'

b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN4853_batch2_white_ok_0.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN5018_batch2_white_ok_0.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG794_batch2_brown_ok_1.jpg'
(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.3137255 , 0.2627451 , 0.13725491, ..., 0.04313726,
         0.03921569, 0.03921569],
        [0.27058825, 0.21960784, 0.09411765, ..., 0.07450981,
         0.07450981, 0.07843138],
        [0.2509804 , 0.20784314, 0.07843138, ..., 0.07450981,
         0.07843138, 0.07058824],
        ...,
        [0.41960785, 0.34901962, 0.

b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG428_batch2_brown_ok_2.jpg'b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG155_batch2_brown_ok_0.jpg'
(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.27058825, 0.24705882, 0.07450981, ..., 0.14509805,
         0.2       , 0.13725491],
        [0.30980393, 0.2784314 , 0.11764706, ..., 0.21176471,
         0.26666668, 0.21568628],
        [0.29411766, 0.2509804 , 0.09803922, ..., 0.1254902 ,
         0.1882353 , 0.1764706 ],
        ...,
        [0.27450982, 0.31764707, 0.24705882, ..., 0.05098039,
         0.04705882, 0.02745098],
        [0.39607844, 0.42352942, 0.32156864, ..., 0.01960784,
         0.01568628, 0.00392157],
        [0.27058825, 0.30588236, 0.2       , ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)>,)
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SN

b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SS6394_batch2_white_ok_0.jpg'(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.3137255 , 0.2784314 , 0.15686275, ..., 0.09803922,
         0.13333334, 0.1254902 ],
        [0.27450982, 0.23921569, 0.10588235, ..., 0.07843138,
         0.10980392, 0.10980392],
        [0.27450982, 0.23921569, 0.09411765, ..., 0.07843138,
         0.10196079, 0.09411765],
        ...,
        [0.654902  , 0.56078434, 0.3764706 , ..., 0.43137255,
         0.3882353 , 0.3647059 ],
        [0.7019608 , 0.63529414, 0.4627451 , ..., 0.4117647 ,
         0.44705883, 0.44705883],
        [0.60784316, 0.5137255 , 0.40392157, ..., 0.28627452,
         0.33333334, 0.32941177]]], dtype=float32)>,)

b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023SG756_batch2_brown_ok_3.jpg'
b'I:\\download\\deepLearning\\datasets\\keelYolo\\keelTwoCalssesDataset\\images\\edged_02023

(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.2509804 , 0.23137255, 0.07843138, ..., 0.22745098,
         0.27450982, 0.22352941],
        [0.3019608 , 0.2627451 , 0.09803922, ..., 0.22745098,
         0.2901961 , 0.25882354],
        [0.2901961 , 0.25490198, 0.09411765, ..., 0.24313726,
         0.28627452, 0.2627451 ],
        ...,
        [0.43137255, 0.41568628, 0.19607843, ..., 0.11372549,
         0.07843138, 0.03921569],
        [0.4392157 , 0.43529412, 0.22745098, ..., 0.16862746,
         0.11764706, 0.06666667],
        [0.47843137, 0.46666667, 0.23529412, ..., 0.22352941,
         0.1882353 , 0.11764706]]], dtype=float32)>,)
(<tf.Tensor: shape=(1, 912, 912), dtype=float32, numpy=
array([[[0.2       , 0.20392157, 0.05882353, ..., 0.10196079,
         0.14509805, 0.09803922],
        [0.2       , 0.2       , 0.05098039, ..., 0.16078432,
         0.20392157, 0.14117648],
        [0.2       , 0.18431373, 0.04705882, ..., 0.10588235,
         0.14117648, 0.07

In [87]:
def image_normalization(img, img_min=0, img_max=255):
    """This is a typical image normalization function
    where the minimum and maximum of the image is needed
    source: https://en.wikipedia.org/wiki/Normalization_(image_processing)
    :param img: an image could be gray scale or color
    :param img_min:  for default is 0
    :param img_max: for default is 255
    :return: a normalized image, if max is 255 the dtype is uint8
    """
    img = np.float32(img)
    epsilon=1e-12 # whenever an inconsistent image
    img = (img-np.min(img))*(img_max-img_min)/((np.max(img)-np.min(img))+epsilon)+img_min
    return img

In [100]:

n=0
#for z in dataset_:
   # n=n+1
input_path = os.path.join(workPath, "edged_02023SG3_batch2_brown_ok_0.jpg")
image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

image = image/255.0 # normalization
image = image.astype(np.float32)
image = cv2.resize(image, (912,912))
image = np.expand_dims(image, axis=-1)
#plt.imshow(image)

res_pre = model.predict(image)
#res = res_pre[-1,:,:]
res *= 255.0
plt.imshow(res)
    #fullpath = os.path.join(outPath, 'masked_'+str(n))
    #cv2.imwrite(fullpath, res_pre)


InvalidArgumentError: Graph execution error:

transpose expects a vector of size 3. But input(1) is a vector of size 4
	 [[{{node model/conv2d/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer}}]] [Op:__inference_predict_function_41526]